Import Library

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from scipy.stats import linregress

Import The Dataset

In [ ]:
data = pd.read_csv('climate data.csv')

Data Preprocessing

In [ ]:
data.head()

In [ ]:
data.columns

In [ ]:
data.shape

In [ ]:
data['DATE'] = pd.to_datetime(data['DATE'])

In [ ]:
numeric_columns = data.select_dtypes(include=[np.number]).columns
data[numeric_columns] = data[numeric_columns].fillna(data[numeric_columns].mean())

In [ ]:
categorical_columns = data.select_dtypes(include=[object]).columns
for column in categorical_columns:
    data[column] = data[column].fillna(data[column].mode()[0])

In [ ]:
data['Sunrise'] = pd.to_datetime(data['Sunrise'], errors='coerce').fillna(method='ffill')
data['Sunset'] = pd.to_datetime(data['Sunset'], errors='coerce').fillna(method='ffill')

In [ ]:
data['WindEquipmentChangeDate'] = pd.to_datetime(data['WindEquipmentChangeDate'], errors='coerce').fillna(pd.Timestamp('1900-01-01'))

In [ ]:
data['WindEquipmentChangeDate'] = pd.to_datetime(data['WindEquipmentChangeDate'], errors='coerce').fillna(pd.Timestamp('1900-01-01'))

In [ ]:
# Verify data types
data.dtypes

In [ ]:
# Display the first few rows after preprocessing
data.head()

Exploratory Data Analysis

In [ ]:
data.describe()

In [ ]:
missing_values = data.isnull().sum()
print("Missing values per column:\n", missing_values)

In [ ]:
missing_percentage = (data.isnull().sum() / len(data)) * 100
print("\nPercentage of missing values per column:\n", missing_percentage)

In [ ]:
missing_summary = pd.DataFrame({
      'Missing Values': missing_values,
          'Percentage (%)': missing_percentage
          })

print("\nSummary of missing data:\n")
print(missing_summary)

In [ ]:
missing_summary.to_csv("missing_values_summary.csv", index=True)

In [ ]:
duplicate_count = data.duplicated().sum()
print(f"Total duplicate rows: {duplicate_count}")

In [ ]:
if duplicate_count > 0:
      print("\nDuplicate rows:\n", data[data.duplicated()])


In [ ]:
plt.figure(figsize=(14, 8))

In [ ]:
# Daily Average Dry Bulb Temperature over time
plt.subplot(3, 1, 1)
plt.plot(data['DATE'], data['DailyAverageDryBulbTemperature'], label='Avg Dry Bulb Temp')
plt.title('Daily Average Dry Bulb Temperature Over Time')
plt.xlabel('Date')
plt.ylabel('Temperature (°C)')
plt.legend()

In [ ]:
# Daily Precipitation over time
plt.subplot(3, 1, 2)
plt.plot(data['DATE'], data['DailyPrecipitation'], label='Daily Precipitation', color='green')
plt.title('Daily Precipitation Over Time')
plt.xlabel('Date')
plt.ylabel('Precipitation (mm)')
plt.legend()

In [ ]:
# Daily Average Wind Speed over time
plt.subplot(3, 1, 3)
plt.plot(data['DATE'], data['DailyAverageWindSpeed'], label='Avg Wind Speed', color='red')
plt.title('Daily Average Wind Speed Over Time')
plt.xlabel('Date')
plt.ylabel('Wind Speed (m/s)')
plt.legend()

plt.tight_layout()
plt.show()

Data Visualisation

In [ ]:
# 1. Line Plots for Time Series Data
plt.figure(figsize=(14, 8))

# Daily Average Dry Bulb Temperature over time
plt.subplot(3, 1, 1)
plt.plot(data['DATE'], data['DailyAverageDryBulbTemperature'], label='Avg Dry Bulb Temp')
plt.title('Daily Average Dry Bulb Temperature Over Time')
plt.xlabel('Date')
plt.ylabel('Temperature (°C)')
plt.legend()



In [ ]:

# Daily Precipitation over time
plt.subplot(3, 1, 2)
plt.plot(data['DATE'], data['DailyPrecipitation'], label='Daily Precipitation', color='green')
plt.title('Daily Precipitation Over Time')
plt.xlabel('Date')
plt.ylabel('Precipitation (mm)')
plt.legend()




In [ ]:
# Daily Average Wind Speed over time
plt.subplot(3, 1, 3)
plt.plot(data['DATE'], data['DailyAverageWindSpeed'], label='Avg Wind Speed', color='red')
plt.title('Daily Average Wind Speed Over Time')
plt.xlabel('Date')
plt.ylabel('Wind Speed (m/s)')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# 4. Scatter Plots for Examining Relationships
# Example: Daily Average Dry Bulb Temperature vs. Daily Precipitation
plt.figure(figsize=(8, 6))
plt.scatter(data['DailyAverageDryBulbTemperature'], data['DailyPrecipitation'], alpha=0.5)
plt.title('Daily Average Dry Bulb Temperature vs. Daily Precipitation')
plt.xlabel('Daily Average Dry Bulb Temperature (°C)')
plt.ylabel('Daily Precipitation (mm)')
plt.show()

# Example: Daily Average Dry Bulb Temperature vs. Daily Average Wind Speed
plt.figure(figsize=(8, 6))
plt.scatter(data['DailyAverageDryBulbTemperature'], data['DailyAverageWindSpeed'], alpha=0.5, color='red')
plt.title('Daily Average Dry Bulb Temperature vs. Daily Average Wind Speed')
plt.xlabel('Daily Average Dry Bulb Temperature (°C)')
plt.ylabel('Daily Average Wind Speed (m/s)')
plt.show()

Trend Analysis

In [ ]:
# Convert DATE column to datetime
data['DATE'] = pd.to_datetime(data['DATE'])

In [ ]:
# Ensure there are no missing values in the relevant column
data['DailyAverageDryBulbTemperature'].fillna(method='ffill', inplace=True)

In [ ]:
# Trend Analysis: Rolling mean of Daily Average Temperature
data['RollingMeanTemperature'] = data['DailyAverageDryBulbTemperature'].rolling(window=365, center=True).mean()

plt.figure(figsize=(14, 7))
plt.plot(data['DATE'], data['DailyAverageDryBulbTemperature'], alpha=0.5, label='Daily Average Temperature')
plt.plot(data['DATE'], data['RollingMeanTemperature'], color='red', label='Rolling Mean Temperature (1 year)')
plt.xlabel('Date')
plt.ylabel('Temperature (°F)')
plt.title('Daily Average Temperature and Rolling Mean Over Time')
plt.legend()
plt.show()


In [ ]:
# Polynomial Fitting
# Fit a polynomial of degree 2 (quadratic) to the temperature data
poly_degree = 2
x = np.arange(len(data))
y = data['DailyAverageDryBulbTemperature'].values
p = np.poly1d(np.polyfit(x, y, poly_degree))

In [ ]:
# Generate polynomial trend line
data['PolynomialTrend'] = p(x)

plt.figure(figsize=(14, 7))
plt.plot(data['DATE'], data['DailyAverageDryBulbTemperature'], alpha=0.5, label='Daily Average Temperature')
plt.plot(data['DATE'], data['PolynomialTrend'], color='green', label=f'Polynomial Trend (degree {poly_degree})')
plt.xlabel('Date')
plt.ylabel('Temperature (°F)')
plt.title('Daily Average Temperature with Polynomial Trend')
plt.legend()
plt.show()

In [ ]:
# Linear Regression Trend Analysis
slope, intercept, r_value, p_value, std_err = linregress(x, y)
data['LinearTrend'] = intercept + slope * x

plt.figure(figsize=(14, 7))
plt.plot(data['DATE'], data['DailyAverageDryBulbTemperature'], alpha=0.5, label='Daily Average Temperature')
plt.plot(data['DATE'], data['LinearTrend'], color='orange', label='Linear Trend')
plt.xlabel('Date')
plt.ylabel('Temperature (°F)')
plt.title('Daily Average Temperature with Linear Trend')
plt.legend()
plt.show()


In [ ]:
# Print Linear Regression Details
print(f"Slope: {slope:.4f}, Intercept: {intercept:.4f}")
print(f"R-squared: {r_value**2:.4f}, P-value: {p_value:.4e}")

In [ ]:
from sklearn.feature_selection import SelectKBest, f_classif, mutual_info_classif
from sklearn.ensemble import RandomForestClassifier